In [5]:
import numpy as np
import pandas as pd
import math

from tqdm import tqdm

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import random
random.seed(141)
np.random.seed(141)

# Problem data

In [37]:
class TwoStateMDP:
    def __init__(self):
        self.num_states = 2
        self.v_actions = [[0,1], [2]] # eligible actions: state0 -> {0 and 1}, state1 ->{2}

        flat_acList = [item for sublist in self.v_actions for item in sublist] # flatten the list of lists
        self.i_numActions = len(set(flat_acList))

        # transition probabilities
        self.v_trp = [[[0 for i in range(self.num_states)] for j in range(self.num_states)] for k in range(self.i_numActions)]

        self.v_trp[0][0][0] = 0.25
        self.v_trp[0][0][1] = 0.75

        self.v_trp[1][0][0] = 0
        self.v_trp[1][0][1] = 1.0

        self.v_trp[2][1][0] = 0
        self.v_trp[2][1][1] = 1.0

        # rewards
        self.v_rews = [[0 for i in range(self.i_numActions)] for j in range(self.num_states)]
        self.v_rews[0][0] = 5
        self.v_rews[0][1] = 10
        self.v_rews[1][2] = -1

In [15]:
mdpEx = TwoStateMDP()
mdpEx.v_trp

[[[0.25, 0.75], [0, 0]], [[0, 1.0], [0, 0]], [[0, 0], [0, 1.0]]]

In [25]:
class InventoryManagement:
    def __init__(self):
        # Mapping of states to indices
        self.state_mapping = {'full': 0, 'medium': 1, 'low': 2, 'empty': 3}
        # Mapping of actions to indices
        self.action_mapping = {'NO': 0, 'OS': 1, 'OM': 2, 'OL': 3}
        # Number of states and actions
        self.num_states = len(self.state_mapping)
        self.num_actions = len(self.action_mapping)
        self.v_actions = [[action for action in range(self.num_actions)] for _ in range(self.num_states)]

        # Initialize transition probabilities and rewards matrices
        self.trp = [[[0.0 for _ in range(self.num_states)] for _ in range(self.num_states)] for _ in range(self.num_actions)]
        self.rews3D = [[[0 for _ in range(self.num_states)] for _ in range(self.num_states)] for _ in range(self.num_actions)]
        # Initialize rewards for actions in all states
        self.rews = [[0 for _ in range(self.num_actions)] for _ in range(self.num_states)]
        
    def load_transitions_and_rewards(self, datafile):
        data = pd.read_csv(datafile)
        # Loop over each row in the DataFrame to populate the matrices
        for _, row in data.iterrows():
            # Get the state, action, and next state indices
            s_idx = self.state_mapping[row['s']]
            a_idx = self.action_mapping[row['a']]
            s_next_idx = self.state_mapping[row['s_next']]
            # Populate the transition probability
            self.trp[a_idx][s_idx][s_next_idx] = row['p(s_next|s,a)']
            # Populate the reward
            self.rews3D[a_idx][s_idx][s_next_idx] = row['r(s,a,s_next)']
        
        # Calculate the expected rewards for actions in all states
        for a in range(self.num_actions):
            for s in range(self.num_states):
                # Expected reward is the sum of rewards weighted by their probabilities
                self.rews[s][a] = sum(self.trp[a][s][ss] * self.rews3D[a][s][ss] for ss in range(self.num_states))

In [26]:
# Create an instance of the class
invEx = InventoryManagement()

# Load the transitions and rewards from the CSV file data
invEx.load_transitions_and_rewards("invData.csv")

# Check if the loading was done correctly
invEx.v_actions, invEx.trp, invEx.rews3D, invEx.rews

([[0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]],
 [[[0.2, 0.8, 0.0, 0.0],
   [0.0, 0.3, 0.6, 0.1],
   [0.0, 0.0, 0.0, 1.0],
   [0.0, 0.0, 0.0, 1.0]],
  [[1.0, 0.0, 0.0, 0.0],
   [0.7, 0.3, 0.0, 0.0],
   [0.0, 1.0, 0.0, 0.0],
   [0.0, 0.0, 1.0, 0.0]],
  [[1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 0.0],
   [0.8, 0.2, 0.0, 0.0],
   [0.0, 1.0, 0.0, 0.0]],
  [[1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 0.0]]],
 [[[5, 5, 0, 0], [0, 2, 2, 2], [0, 0, 0, -10], [0, 0, 0, -20]],
  [[-2, 0, 0, 0], [4, 4, 0, 0], [0, 0, 0, 0], [0, 0, -15, 0]],
  [[-2, 0, 0, 0], [-1, 0, 0, 0], [10, 10, 0, 0], [0, -5, 0, 0]],
  [[-2, 0, 0, 0], [-1, 0, 0, 0], [-1, 0, 0, 0], [15, 0, 0, 0]]],
 [[5.0, -2.0, -2.0, -2.0],
  [1.9999999999999998, 4.0, -1.0, -1.0],
  [-10.0, 0.0, 10.0, -1.0],
  [-20.0, -15.0, -5.0, 15.0]])

# Policy Evaluation Algorithm

In [9]:
def PolicyEval_VI(giNumStates, gvPol, gvTrp, gvRews, gdDiscountFactor=0.9):
    
    print("Policy",gvPol)

    t_dEpsilon = 0.0001
    tmpv_vals = [0]*giNumStates
    
    t_iCtr = 0
    while True:
        # print("t_iCtr", t_iCtr)
        # print("tmpv_vals", tmpv_vals)
        currentv_vals = [0]*giNumStates
        for s in range(giNumStates):
            
            a = gvPol[s]
            currentv_vals[s] = gvRews[s][a]

            for ss in range(giNumStates):
                currentv_vals[s] += gdDiscountFactor*gvTrp[a][s][ss]*tmpv_vals[ss]
        
        currentv_vals = np.around(currentv_vals, 3)
        # print("currentv_vals", currentv_vals)
        if np.linalg.norm(np.array(tmpv_vals) - np.array(currentv_vals)) < t_dEpsilon:
            break
        
        tmpv_vals = currentv_vals
        t_iCtr = t_iCtr + 1

    return tmpv_vals, t_iCtr

In [16]:
v_vals,_ = PolicyEval_VI(mdpEx.num_states, [0,2], mdpEx.v_trp, mdpEx.v_rews, gdDiscountFactor=0.9)

print('\nvalues at each state')
for i in range(len(v_vals)):
    print("V(",i,")", v_vals[i])

Policy [0, 2]

values at each state
V( 0 ) -2.254
V( 1 ) -9.996


In [18]:
# inventory ex: evaluate the policy of always order large (OL)
v_vals,_ = PolicyEval_VI(invEx.num_states, [3,3,3,3], invEx.trp, invEx.rews, gdDiscountFactor=0.9)

print('\nvalues at each state')
for i in range(len(v_vals)):
    print("V(",i,")", v_vals[i])

Policy [3, 3, 3, 3]

values at each state
V( 0 ) -19.996
V( 1 ) -18.996
V( 2 ) -18.996
V( 3 ) -2.996


In [20]:
# inventory ex: evaluate the policy of ordering based on inventory level, e.g., OL when empty, NO when full
v_vals,_ = PolicyEval_VI(invEx.num_states, [0,1,2,3], invEx.trp, invEx.rews, gdDiscountFactor=0.9)

print('\nvalues at each state')
for i in range(len(v_vals)):
    print("V(",i,")", v_vals[i])

Policy [0, 1, 2, 3]

values at each state
V( 0 ) 45.032
V( 1 ) 44.342
V( 2 ) 50.405
V( 3 ) 55.529


# Value Iteration Algorithm

In [21]:
def ValueIteration(giNumStates, gvActions, gvTrp, gvRews, gdDiscountFactor=0.9):
    
    t_dEpsilon = 0.000001
    tmpv_vals = [0]*giNumStates

    flat_list = [item for sublist in gvActions for item in sublist] # flatten the list of lists
    t_iNumActions = len(set(flat_list))
    
    t_iCtr = 0
    while True:
        
        # print("t_iCtr", t_iCtr)
        # print("tmpv_vals", tmpv_vals)
        
        currentv_vals = [0]*giNumStates
        t_qValues = [[float("-inf") for a in range(t_iNumActions)] for s in range(giNumStates)]
        t_vPol = [float("-inf") for s in range(giNumStates)]
        for s in range(giNumStates):
            for a in gvActions[s]:
                t_qValues[s][a] = gvRews[s][a]
                for ss in range(giNumStates):
                    t_qValues[s][a] += gdDiscountFactor*gvTrp[a][s][ss]*tmpv_vals[ss]
        
        # print("t_qValues", t_qValues)
        currentv_vals = np.amax(np.around(t_qValues, 3), axis = 1)
        t_vPol = np.argmax(np.around(t_qValues, 3), axis = 1)
        # print("currentv_vals", currentv_vals)
        if np.linalg.norm(np.array(tmpv_vals) - np.array(currentv_vals)) < t_dEpsilon:
            break
        
        tmpv_vals = currentv_vals
        t_iCtr = t_iCtr + 1

    return tmpv_vals, t_vPol, t_iCtr

In [28]:
v_vals, t_vPol, _ = ValueIteration(mdpEx.num_states, mdpEx.v_actions, mdpEx.v_trp, mdpEx.v_rews, gdDiscountFactor=0.9)
for i in range(len(v_vals)):
    print("V(",i,")", v_vals[i])

V( 0 ) 1.004
V( 1 ) -9.996


In [33]:
v_vals, v_pol, _ = ValueIteration(invEx.num_states, invEx.v_actions, invEx.trp, invEx.rews, gdDiscountFactor=0.9)
print("\nvalues-policy:\n")
for val in zip(v_vals, v_pol):
    print(val)


values-policy:

(52.39, 0)
(52.722, 0)
(57.211, 2)
(62.151, 3)


# QLearning

In [60]:
def QLearning(num_states, num_actions, v_actions, v_trp, v_rews, episodes = 100, runs=10, discount_factor=0.9, step_size=0.5):
    iter_limit = 100
    epsilon = 0.05  # randomness in action selection

    rewards_q_learning = np.zeros(episodes)
    for r in tqdm(range(runs)):
        # initialize Q-values
        q_value = np.full((num_states, num_actions), -np.inf)  # Use negative infinity for impossible actions
        for s in range(num_states):
            for a in v_actions[s]:
                q_value[s, a] = 0.0  # Initialize eligible actions to zero

        for i in range(episodes):
            state = random.choice(range(num_states))  # Start at a random state
            rewards = 0.0

            for iter_ctr in range(iter_limit):
                # Epsilon-greedy action selection
                if random.random() < epsilon:
                    action = np.random.choice(v_actions[state])
                else:
                    # Select one of the actions with the highest Q-value
                    max_q_value = np.max(q_value[state, v_actions[state]])
                    best_actions = [a for a in v_actions[state] if q_value[state, a] == max_q_value]
                    action = np.random.choice(best_actions)

                # Transition to next state and receive reward
                # Here we assume that the transition probabilities for each action are stored in a way that corresponds to the state indices.
                next_state = np.random.choice(np.arange(num_states), p=v_trp[action][state])
                reward = v_rews[state][action]
                rewards += reward

                # Q-Learning update
                q_value[state, action] += step_size * (reward + discount_factor * np.max(q_value[next_state, v_actions[next_state]]) - q_value[state, action])

                state = next_state

            rewards_q_learning[i] += rewards

    rewards_q_learning /= runs
    plt.plot(rewards_q_learning, label='Q-Learning')
    plt.xlabel('Episodes')
    plt.ylabel('Sum of rewards during episode')
    plt.legend()
    plt.show()

    # Derive the optimal policy from Q-values
    optimal_policy = {s: np.argmax(q_value[s, :]) for s in range(num_states) if not np.all(q_value[s, :] == -np.inf)}

    return optimal_policy, q_value

In [61]:
optimal_policy, q_value = QLearning(mdpEx.num_states, mdpEx.i_numActions, mdpEx.v_actions, mdpEx.v_trp, mdpEx.v_rews)

print("\nvalues-policy:\n")
for val in zip(np.max(q_value,1), optimal_policy):
    print(val)

100%|██████████| 10/10 [00:01<00:00,  5.69it/s]


values-policy:

(1.0000000000000193, 0)
(-9.999999999999979, 1)



/var/folders/q8/jq2qqhms2y18nl_r3vs5007r0000gn/T/ipykernel_9076/649298873.py:45: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [62]:
optimal_policy, q_value = QLearning(invEx.num_states, invEx.num_actions, invEx.v_actions, invEx.trp, invEx.rews, episodes = 1000, runs=10)

print("\nvalues-policy:\n")
for val in zip(np.max(q_value,1), optimal_policy):
    print(val)

100%|██████████| 10/10 [00:17<00:00,  1.76s/it]


values-policy:

(54.348122687990305, 0)
(55.01744333343609, 1)
(59.17852384051515, 2)
(64.25363273966377, 3)



/var/folders/q8/jq2qqhms2y18nl_r3vs5007r0000gn/T/ipykernel_9076/649298873.py:45: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
